In [22]:
import os
import pyodbc
import pandas as pd
import numpy as np
import shutil

In [23]:
server = 'tcp:sqlserverprod'
database = 'Reporting'
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')
cursor = conn.cursor()

In [24]:
UnProcessedPath = os.path.join('C:', os.sep, 'FilePath','UnProcessed')
ProcessedPath = os.path.join('C:', os.sep, 'FilePath','Processed')

In [25]:
with os.scandir(UnProcessedPath) as files:
    for file in files:
        if file.is_file():
            xlsfile=file.name

In [26]:
unprocessedfilepath = os.path.join(UnProcessedPath, xlsfile)
processedfilepath = os.path.join(ProcessedPath, xlsfile)
unprocessedfilepath

'C:\\FilePath\\UnProcessed\\Power query 6-12 Jan 2020.xlsx'

In [27]:
df_xlsx = pd.read_excel(unprocessedfilepath)
cleandf = df_xlsx.dropna(how='all')

In [28]:
non_nan_df = cleandf.replace(np.nan,'',regex=True)
non_nan_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1503 entries, 0 to 1502
Data columns (total 22 columns):
First five        1503 non-null object
No space          1503 non-null object
Correct unit#     1503 non-null object
Truck Status      1503 non-null object
Division          1503 non-null object
Truck Class       1503 non-null object
Truck ID          1503 non-null object
Post_Date         1503 non-null object
Invoice_Date      1503 non-null object
Source            1503 non-null object
Read_Type         1503 non-null object
PrePass_Id        1503 non-null float64
EZPassId_Plate    1503 non-null object
Truck_Id          1503 non-null object
Agency            1503 non-null object
Entry_Plaza       1503 non-null object
Entry_Date        1503 non-null object
Exit_Plaza        1503 non-null object
Exit_Date         1503 non-null object
Class             1503 non-null float64
Miles             1503 non-null object
Toll_Charge       1503 non-null float64
dtypes: float64(3), object(19)
m

In [29]:
non_nan_df[['Truck ID', 'PrePass_Id']] = non_nan_df[['Truck ID', 'PrePass_Id']].apply(pd.to_numeric)
non_nan_df.info()
non_nan_df.to_excel(ProcessedPath+os.sep+'test.xlsx', index=False)

#non_nan_df['Truck ID']=non_nan_df.astype({'Truck ID':'int32'}).dtypes
#non_nan_df['Invoice_Date']=non_nan_df['Invoice_Date'].astype(str)
#non_nan_df['PrePass_Id']=non_nan_df['PrePass_Id'].astype('int64')
#non_nan_df['Entry_Date']=non_nan_df['Entry_Date'].astype(str)
#non_nan_df['Class']=non_nan_df['Class'].astype(str)
#non_nan_df['Miles']=non_nan_df['Miles'].astype(str)
#non_nan_df['Toll_Charge']=non_nan_df['Toll_Charge'].astype(str)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1503 entries, 0 to 1502
Data columns (total 22 columns):
First five        1503 non-null object
No space          1503 non-null object
Correct unit#     1503 non-null object
Truck Status      1503 non-null object
Division          1503 non-null object
Truck Class       1503 non-null object
Truck ID          1025 non-null float64
Post_Date         1503 non-null object
Invoice_Date      1503 non-null object
Source            1503 non-null object
Read_Type         1503 non-null object
PrePass_Id        1503 non-null float64
EZPassId_Plate    1503 non-null object
Truck_Id          1503 non-null object
Agency            1503 non-null object
Entry_Plaza       1503 non-null object
Entry_Date        1503 non-null object
Exit_Plaza        1503 non-null object
Exit_Date         1503 non-null object
Class             1503 non-null float64
Miles             1503 non-null object
Toll_Charge       1503 non-null float64
dtypes: float64(4), object(18)


In [21]:
for index, row in non_nan_df.iterrows():
    cursor.execute("INSERT INTO [dbo].[TexasToll]([First five],[No space],[Correct unit#],[Truck Status],[Division],[Truck Class],[Truck ID],[Post_Date],[Invoice_Date],[Source],[Read_Type],[PrePass_Id],[EZPassId_Plate],[Truck_Id],[Agency],[Entry_Plaza],[Entry_Date],[Exit_Plaza],[Exit_Date],[Class],[Miles],[Toll_Charge]) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", row['First five'], row['No space'], row['Correct unit#'], row['Truck Status'], row['Division'], row['Truck Class'], row['Truck ID'], row['Post_Date'], row['Invoice_Date'], row['Source'], row['Read_Type'], row['PrePass_Id'], row['EZPassId_Plate'], row['Truck_Id'], row['Agency'], row['Entry_Plaza'], row['Entry_Date'], row['Exit_Plaza'], row['Exit_Date'], row['Class'], row['Miles'], row['Toll_Charge'])

conn.commit()
cursor.close()
conn.close()

ProgrammingError: ('42000', '[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The incoming tabular data stream (TDS) remote procedure call (RPC) protocol stream is incorrect. Parameter 10 (""): The supplied value is not a valid instance of data type float. Check the source data for invalid values. An example of an invalid value is data of numeric type with scale greater than precision. (8023) (SQLExecDirectW)')

In [ ]:
#shutil.move(unprocessedfilepath, processedfilepath)